# Querying Data from `FTA Bus Award StoryMap`

Currently FTA uses ArcGIS storymap to display their bus award data. Is is possible to extract this data using their REST server.
Extracting bus award data from a server will be the most reliable way to see the data.

In [1]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [2]:
import _bus_cost_utils
from calitp_data_analysis.sql import to_snakecase

# Get geojson from ArcGIS Rest Server 

https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query
https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query


# Steps to Query Rest Server

1. Set `Where:` field to `1=1`
2. Set `Out Field:` field to `*`
3. Set `Format:` to `Geojson`
4. Click `Query(GET)`
5. Copy paste given URL 

In [3]:
fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="


fy_24_data = to_snakecase(gpd.read_file(fy24))
fy_23_data = to_snakecase(gpd.read_file(fy23))

display(
    fy_24_data.shape,
    type(fy_24_data),
    fy_24_data.info(),
    fy_23_data.shape,
    type(fy_23_data),
    fy_23_data.info(),
)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   application_number_       117 non-null    object  
 1   agency_name               117 non-null    object  
 2   low_no_bbf                117 non-null    object  
 3   project_description       117 non-null    object  
 4   number_of_buses_          117 non-null    int64   
 5   funding_amount            116 non-null    float64 
 6   project_title             117 non-null    object  
 7   project_type              117 non-null    object  
 8   display_type              117 non-null    object  
 9   overall_propulsion_type   117 non-null    object  
 10  specific_propulsion_type  117 non-null    object  
 11  areas_served              117 non-null    object  
 12  congressional_districts   117 non-null    object  
 13  address_line_1            117 non-null    

(117, 22)

geopandas.geodataframe.GeoDataFrame

None

(130, 25)

geopandas.geodataframe.GeoDataFrame

None

# Explore FY23 data

In [4]:
fy_23_data = fy_23_data.assign(
    extracted_prop_type=fy_23_data["project_description"].apply(
        _bus_cost_utils.new_prop_finder
    ),
    extracted_bus_size=fy_23_data["project_description"].apply(
        _bus_cost_utils.new_bus_size_finder
    ),
    fy="fy23",
)

In [5]:
fy_23_data["total_bus_count"] = fy_23_data[
    ["traditional_buses", "low_emission_buses", "zero_emission_buses"]
].sum(axis=1)

In [6]:
display(fy_23_data.columns, fy_23_data.head(2))

Index(['application_number', 'fta_region', 'project_sponsor', 'project_title',
       'project_description', 'original_statement_of_work', 'funding',
       'number_of_buses', 'traditional_buses', 'low_emission_buses',
       'zero_emission_buses', 'project_type', 'display_type', 'areas_served',
       'congressional_districts', 'propulsion_type', 'address1', 'address2',
       'city', 'state', 'zip', 'x', 'y', 'objectid', 'geometry',
       'extracted_prop_type', 'extracted_bus_size', 'fy', 'total_bus_count'],
      dtype='object')

,application_number,fta_region,project_sponsor,project_title,project_description,original_statement_of_work,funding,number_of_buses,traditional_buses,low_emission_buses,zero_emission_buses,project_type,display_type,areas_served,congressional_districts,propulsion_type,address1,address2,city,state,zip,x,y,objectid,geometry,extracted_prop_type,extracted_bus_size,fy,total_bus_count
0,BUSFAC-2023-3691,8,Utah Department of Transportation on behalf of Park City Transit,"Park City's Comprehensive Bus Stop Redesign and Rehabilitation: Increasing Efficiency, Improving ADA Compliance and Preparing for the Expansion of Affordable Housing Opportunities.","The Utah Department of Transportation will receive funding on behalf of Park City Transit to improve more than 72 bus stops, including rider amenities. The project will provide a safer, more appealing interface with riders, particularly in areas with historically disadvantaged communities as well as seasonal visitors.","Through this project, Park City Transit will engage in the design, construction, and installation of improved passenger amenities at 72 individual bus stops within its service area. Of particular emphasis will be the provision of 12 upgraded bus stop sites serving a new affordable housing community of 500 units as well as enhancements to existing SR-248 corridor stops in areas previously identified as historically disadvantaged populations with an elevated degree of transit dependency.","$7,393,183",0,0,0,0,Facility,Zero Facility,Rural,UT-001,Zero / Traditional,4501 Constitution Blvd,NaN,Taylorsville,UT,84129,-111.956609,40.671120,1,POINT (-111.95661 40.67112),not specified,not specified,fy23,0
1,BUSFAC-2023-3715,9,Northern Arizona Intergovernmental Public Transportation Authority,"Maintenance Facility Improvements: Essential project to reach community goals for climate action, racial equity, transition to electric buses, and improve mechanic safety and efficiency.","The Northern Arizona Intergovernmental Public Transportation Authority (Mountain Line) will receive funding to build a maintenance facility at the agency's headquarters. This building will house the entire bus fleet, improve maintenance and create jobs in zero-emission technology as it strives to better serve the Flagstaff community.","Funding supports the construction of the maintenance facility at our headquarters. Mountain Line has received $1,600,000 in a competitive FTA Arizona 5339/5307 grant for the design of this facility. The existing facility is not designed for 60 buses, only fitting half the bus inside and is not tall enough to safely perform work on batteries nor has the required electric outlets for electric buses. This project improves fleet reliability by reducing mechanic down time, create good-paying jobs in zero emission technology, modernize our campus and enable Mountain Line to meet growth assumptions that connect communities. Mountain Line completes all maintenance in-house and has high standards to maintain a state of good repair for all fleet, as demonstrated by the fact that Mountain Line has never had a maintenance finding in an FTA triennial review. This project takes a proactive approach to addressing maintenance issues to maintain our high standards for state of good repair.","$16,358,000",0,0,0,0,Facility,Zero Facility,Small Urban,AZ-002,Zero / Low,3773 N Kaspar Drive,NaN,Flagstaff,AZ,86004,-111.592407,35.218441,2,POINT (-111.59241 35.21844),zero-emission bus (not specified),not specified,fy23,0


In [7]:
# clean the funding columns to remove $ and ,
fy_23_data["funding"] = (
    fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")
)

fy_23_data["zero_emission_buses"] = fy_23_data["zero_emission_buses"].astype("int64")

/tmp/ipykernel_1612/1028622314.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")


In [8]:
fy_23_data["propulsion_type"].value_counts()

Zero                        54
Low                         43
Traditional                 17
Zero / Traditional           5
Zero / Low                   3
Low / Traditional            3
Other                        2
Zero / Low / Traditional     2
Combined                     1
Name: propulsion_type, dtype: int64

In [9]:
fy_23_data["project_type"].value_counts()

Bus                                      40
Facility                                 25
Bus / Chargers                           17
Bus / Facility                           13
Bus / Facility / Chargers                 9
Bus / Equipment                           6
Bus / Chargers / Equipment                5
Facility / Chargers                       3
Bus / Facility / Equipment                2
Chargers                                  2
Bus / Facility / Chargers / Equipment     1
Facility / Equipment                      1
Bus / Chargers / Other                    1
Bus / Facility / Equipment / Other        1
Bus / Other                               1
Chargers / Equipment                      1
Bus / Facility / Other                    1
Facility / Chargers / Equipment           1
Name: project_type, dtype: int64

In [10]:
fy_23_data["project_sponsor"].value_counts().head()

New Mexico Department of Transportation on behalf of North Central Regional Transit District    2
State of California on behalf of Kern Regional Transit                                          2
Regional Public Transportation Authority (Valley Metro)                                         1
Metropolitan Transit Authority of Harris County (METRO)                                         1
State of California on behalf of Glenn County Transportation Commission                         1
Name: project_sponsor, dtype: int64

In [11]:
# filtering down fy23 data to just ZEB rows (by FTA project type and proulsion type)
col_list_23 = [
    # "application_number",
    "project_sponsor",
    "project_title",
    "project_description",
    # "original_statement_of_work",
    "project_type",
    "funding",
    # "number_of_buses",
    #'traditional_buses',
    #'low_emission_buses',
    # "zero_emission_buses",
    "total_bus_count",
    # "propulsion_type",
    "extracted_prop_type",
    "extracted_bus_size",
    # "fy"
]

fy_23_bus = fy_23_data[
    (fy_23_data["project_type"] == "Bus")
    # & (fy_23_data["propulsion_type"] == "Zero")
    # (fy_23_data["low_emission_buses"] == 0) &
    # (fy_23_data["zero_emission_buses"] > 0)
][col_list_23]

fy_23_all_projects = fy_23_data[col_list_23]

In [12]:
fy_23_all_projects.shape

(130, 8)

In [13]:
display(
    fy_23_bus.shape,
    fy_23_bus.dtypes,
    fy_23_bus[fy_23_bus["extracted_prop_type"] == "not specified"],
)

(40, 8)

project_sponsor        object
project_title          object
project_description    object
project_type           object
funding                 int64
total_bus_count         int64
extracted_prop_type    object
extracted_bus_size     object
dtype: object

,project_sponsor,project_title,project_description,project_type,funding,total_bus_count,extracted_prop_type,extracted_bus_size
2,Virginia Department of Rail and Public Transportation (DRPT),VA Rural Transit Asset Management and Equity Program,The Virginia Department of Rail and Public Transportation will receive funding to replace the oldest buses operated by 11 rural transit providers. The project includes the purchase of a few dozen replacement buses that are vital to reducing a backlog of adequate vehicles.,Bus,4690010,39,not specified,not specified
7,South Carolina Department of Transportation on behalf of 25 rural transit providers,SCDOT Vehicle Replacement Project,The South Carolina Department of Transportation will receive funding to buy new replacement vehicles for 25 rural transit agencies throughout the state. This project will improve transit service and reliability for residents in rural South Carolina communities.,Bus,15423904,160,not specified,not specified
17,South Dakota Department of Transportation on behalf of ARL and CTWSI,Replacement of Aberdeen Ride Line fleet buses Replacement and Expansion of Community Transit of Watertown/Sisseton Inc. fleet buses,"The South Dakota Department of Transportation, on behalf of Aberdeen Ride Line and Community Transit of Watertown/Sisseton Inc., will receive funding for nine new alternative, ethanol-fueled buses to reduce greenhouse gas emissions in the city of Aberdeen and 15 counties. The new buses will improve safety and reliability and reduce maintenance costs while also providing a more comfortable passenger ride.",Bus,1006750,9,not specified,not specified
22,Madison County Mass Transit District,Heavy Duty 40-Foot Bus Replacement,"The Madison County Mass Transit District will receive funding to buy buses. This project will improve transit service and reliability for residents in Madison County, Illinois.",Bus,1080000,2,not specified,not specified
26,State of California on behalf of Kern Regional Transit,Purchase of Fifteen (15) Replacement Cutaway Buses,"The State of California, on behalf of Kern Regional Transit, will receive funding to buy cutaway vehicles to replace older vehicles that have exceeded their useful life. The vehicles will be used for both fixed route and paratransit services.",Bus,2932500,15,not specified,cutaway
30,The Colorado Department of Transportation (CDOT) on behalf of San Miguel Authority for Regional Transportation,SMART Vanpool Vehicle Replacement Project,"The Colorado Department of Transportation (CDOT) on behalf of the San Miguel Authority for Regional Transportation in southwest Colorado will receive funding to buy vans to replace older ones. The project will enable the agency to improve its reliability and attract riders in San Miguel, Dolores, Montrose and Ouray counties.",Bus,233760,3,not specified,not specified
45,Illinois Department of Transportation on behalf of 33 subrecipients,Illinois DOT Statewide Paratransit Vehicle Replacements,The Illinois Department of Transportation will receive funding to buy cutaway paratransit buses and vans to replace older vehicles for urban and rural public transportation providers throughout Illinois. This project will enhance transit access and mobility for rural communities around the state.,Bus,12600000,134,not specified,cutaway


In [14]:
# cleaning the "not specified" rows
_bus_cost_utils.col_row_updater(
    fy_23_bus,
    "project_title",
    "VA Rural Transit Asset Management and Equity Program",
    "extracted_prop_type",
    "mix (low emission)",
)

# Explore FY 24 data

overall, not as grandular as the FY 23 data. FY24 is missing some ZEB bus counts, original SOW

In [15]:
fy_24_data = fy_24_data.assign(
    extracted_prop_type=fy_24_data["project_description"].apply(
        _bus_cost_utils.new_prop_finder
    ),
    extracted_bus_size=fy_24_data["project_description"].apply(
        _bus_cost_utils.new_bus_size_finder
    ),
    fy="fy24",
)

In [16]:
list(fy_24_data.columns)

['application_number_',
 'agency_name',
 'low_no_bbf',
 'project_description',
 'number_of_buses_',
 'funding_amount',
 'project_title',
 'project_type',
 'display_type',
 'overall_propulsion_type',
 'specific_propulsion_type',
 'areas_served',
 'congressional_districts',
 'address_line_1',
 'city',
 'state',
 'additional_zip',
 'zip_code',
 'x',
 'y',
 'fid2',
 'geometry',
 'extracted_prop_type',
 'extracted_bus_size',
 'fy']

In [17]:
fy_24_data.head(2)

,application_number_,agency_name,low_no_bbf,project_description,number_of_buses_,funding_amount,project_title,project_type,display_type,overall_propulsion_type,specific_propulsion_type,areas_served,congressional_districts,address_line_1,city,state,additional_zip,zip_code,x,y,fid2,geometry,extracted_prop_type,extracted_bus_size,fy
0,BUSFAC-2024-4750,"Alaska Department of Transportation, on behalf of City and Borough of Juneau, Capital Transit",BBF,"Alaska Department of Transportation, on behalf of Capital Transit, will receive funding to buy new electric buses, install charging equipment, and initiate a workforce development program. The project will improve service, reliability and air quality throughout the city and borough of Juneau in Alaska’s Southeast panhandle.",6,11855112.0,Mitigating Overcrowding and Enhancing Sustainability in Alaskas Capital City,Vehicle | Facility | Chargers,Zero Bus and Facility,Zero,Battery electric,Rural,AK-001,PO Box 112500,Juneau,AK,2500,99811,-134.41630,58.29971,1,POINT (-134.41631 58.29971),electric (not specified),not specified,fy24
1,BUSFAC-2024-5061,City of Montgomery / The M Public Transit System,BBF,"The city of Montgomery's The M transit system will receive funding to buy battery electric buses, charging equipment and to initiate a workforce training program. The project will ensure continued service reliability, maintain a state of good repair and, by improving air quality, advance environmental justice in the Montgomery community.",8,16941377.0,City of Montgomery / The M Public Transit System Electric Bus Program,Vehicle | Chargers,Zero Bus and Facility,Zero,Battery electric,Large Urban,AL-002 | AL-007,103 N. Perry Street,Montgomery,AL,3728,36104,-86.30811,32.37930,2,POINT (-86.30811 32.37930),BEB,not specified,fy24


In [18]:
display(
    fy_24_data["specific_propulsion_type"].value_counts(),
    fy_24_data["project_type"].value_counts(),
    fy_24_data["extracted_prop_type"].value_counts(),
)

Battery electric                                            38
Diesel-electric hybrid                                      14
N/A                                                         12
CNG                                                         11
Gasoline                                                     9
Hydrogen fuel cell                                           8
Diesel                                                       6
Propane                                                      4
Diesel | Gasoline                                            3
Battery electric | Hydrogen fuel cell                        2
Diesel-electric hybrid | Hydrogen fuel cell                  2
Battery electric | Diesel-electric hybrid | Propane          1
CNG | Gasoline                                               1
CNG | Diesel | Diesel-electric hybrid | Gasoline | Other     1
CNG | Diesel-electric hybrid                                 1
Battery electric | Propane                             

Vehicle                           40
Vehicle | Chargers                15
Facility                          15
Vehicle | Facility | Chargers     10
Vehicle | Facility                 7
Vehicle | Equipment                6
Vehicle | Facility | Equipment     5
Vehicle | Facility                 5
Facility | Chargers                4
Facility | Equipment               3
Vehicle | Chargers | Equipment     3
Facility                           2
Vehicle                            2
Name: project_type, dtype: int64

not specified                        31
BEB                                  22
low emission (hybrid)                18
electric (not specified)             13
FCEB                                 12
CNG                                   9
zero-emission bus (not specified)     8
low emission (propane)                4
Name: extracted_prop_type, dtype: int64

In [19]:
# filtering FY24 data to just show ZEB.

col_list_24 = [
    # "application_number_",
    "agency_name",
    "project_title",
    "project_description",
    "project_type",
    "funding_amount",
    "number_of_buses_",
    # "overall_propulsion_type",
    # "specific_propulsion_type",
    "extracted_prop_type",
    "extracted_bus_size",
    # "fy"
]

project_val = ["Vehicle ", "Vehicle"]

prop_vals = [
    "Battery electric",
    "Hydrogen fuel cell",
    "Battery electric | Hydrogen fuel cell",
    "Battery electric ",
]

fy_24_bus = fy_24_data[col_list_24][
    (fy_24_data["project_type"].isin(project_val))
    # & (fy_24_data["specific_propulsion_type"].isin(prop_vals))
]

fy_24_all_projects = fy_24_data[col_list_24]

In [20]:
fy_24_all_projects.shape

(117, 8)

In [21]:
display(
    fy_24_bus.shape,
    fy_24_bus.columns,
)

(42, 8)

Index(['agency_name', 'project_title', 'project_description', 'project_type',
       'funding_amount', 'number_of_buses_', 'extracted_prop_type',
       'extracted_bus_size'],
      dtype='object')

In [22]:
fy_24_bus

,agency_name,project_title,project_description,project_type,funding_amount,number_of_buses_,extracted_prop_type,extracted_bus_size
4,Salt River Pima-Maricopa Indian Community (SRPMIC),Salt River Pima-Maricopa Indian Community Bus and Bus Facility 5339(b) Grant,"The Salt River Pima-Maricopa Indian Community will receive funding to buy accessible vans to replace vehicles nearing the end of their useful life. The vans will provide on-demand service for residents to reach jobs, school, recreation and other essential services throughout Scottsdale, Mesa and Tempe.",Vehicle,425001.0,3,not specified,not specified
6,"California Department of Transportation, on behalf of Redwood Coast Transit Authority",RCTA 'Rust and Rot' Bus Replacement Project,"The California Department of Transportation, on behalf of the Redwood Coast Transit Authority in Crescent City, will receive funding to buy new buses to help replace older buses that have reached the end of their useful life. The buses will create a more reliable fleet and help RCTA avoid service cuts.",Vehicle,474478.0,2,not specified,not specified
7,"California Department of Transportation, on behalf of Humboldt Transit Authority",Project 24-02: Replacement of Class 6 Medium Duty Intercity Cutaways,"The California Department of Transportation, on behalf of the Humboldt Transit Authority in Eureka, CA, will receive funding to buy new buses to replace older buses that have exceeded their useful life. The new buses will ensure residents from southern Humboldt County will continue to have access to northern county services, such as healthcare and other social services.",Vehicle,639000.0,3,not specified,not specified
8,"California Department of Transportation, on behalf of Morongo Basin Transit Authority",Battery Electric Zero Emission Vehicle,"The California Department of Transportation, on behalf of the Morongo Basin Transit Authority, will receive funding to buy a battery-electric bus to replace an older vehicle. This project will improve air quality, safety, reliability and state of good repair in the San Bernadino County service area.",Vehicle,131168.0,1,BEB,not specified
9,Santa Barbara Metropolitan Transit District,The Central Coast Transit Consortiums 'Electrifying Californias Central Coast',"The Santa Barbara Metropolitan Transportation District will receive funding to buy battery-electric buses and chargers to replace older diesel buses, which will improve service reliability and reduce air pollution.",Vehicle,2894131.0,2,electric (not specified),not specified
12,"California Department of Transportation, on behalf of Lassen Transit Service Agency",Purchase Replacement of a 23ft Bus,"The California Department of Transportation, on behalf of Lassen Transit Service Agency, will receive funding to buy a new bus to replace an older, unreliable vehicle. The new bus will allow LTSA to maintain current service levels in rural northeastern Nevada and establish new service to Reno, connecting its growing senior population to essential medical facilities and other services.",Vehicle,154367.0,1,not specified,not specified
20,City of Greeley,"Acquire Five (5) New CNG Buses to Begin Premium Transit Service on US-34 from Greeley, CO to Loveland, CO","The city of Greeley will receive funding to buy compressed natural gas (CNG) buses for use on a new transit service between Greeley and Loveland, CO.",Vehicle,3508404.0,5,CNG,not specified
23,"Colorado Department of Transportation, on behalf of Eagle Valley Transportation Authority",Diesel Fleet Replacement to Improve Service Reliability and Reduce Vehicle Emissions,"The Colorado Department of Transportation, on behalf of Eagle Valley Transportation Authority, will receive funding to buy hybrid-electric buses. The buses will replace older diesel vehicles with new, more reliable vehicles that will provide better transit service and improve regional air quality.",Vehicle,4573000.0,5,low emission (hybrid),not specified
27,"Colorado Department of Transportat

In [23]:
# fixing individual rows
# this is a bandaid until i update the prop finder function

_bus_cost_utils.col_row_updater(
    fy_24_bus, "funding_amount", "2894131.0", "extracted_prop_type", "BEB"
)
_bus_cost_utils.col_row_updater(
    fy_24_bus, "funding_amount", "14415095.0", "extracted_prop_type", "BEB"
)
_bus_cost_utils.col_row_updater(
    fy_24_bus, "funding_amount", "18112632.0", "extracted_prop_type", "BEB"
)

In [24]:
fy_24_bus["extracted_prop_type"].value_counts()

not specified                        12
low emission (hybrid)                12
BEB                                   5
CNG                                   5
electric (not specified)              3
low emission (propane)                3
zero-emission bus (not specified)     1
FCEB                                  1
Name: extracted_prop_type, dtype: int64

# Filtered Dataframes
Filtered to show only ZEB projects for 23 and 24 only. 
Project type is just buses, not facilities, equipment or any combinations including buses

In [25]:
display(
    fy_23_bus.shape,
    list(fy_23_bus.columns),
    fy_24_bus.shape,
    list(fy_24_bus.columns),
)

# 'original_statement_of_work' and 'zero_emission_buses' is not on FY24
# 'specific_propulsion_type' is not in fy23

(40, 8)

['project_sponsor',
 'project_title',
 'project_description',
 'project_type',
 'funding',
 'total_bus_count',
 'extracted_prop_type',
 'extracted_bus_size']

(42, 8)

['agency_name',
 'project_title',
 'project_description',
 'project_type',
 'funding_amount',
 'number_of_buses_',
 'extracted_prop_type',
 'extracted_bus_size']

In [26]:
col_dict = {
    "project_sponsor": "transit_agency",
    "agency_name": "transit_agency",
    "project_type": "new_project_type",
    "funding": "total_cost",
    "funding_amount": "total_cost",
    "total_bus_count": "bus_count",
    "number_of_buses_": "bus_count",
    "extracted_prop_type": "prop_type",
    "extracted_bus_size": "bus_size_type",
}

In [27]:
fy_23_bus = fy_23_bus.rename(columns=col_dict)

In [28]:
fy_24_bus = fy_24_bus.rename(columns=col_dict)

In [29]:
display(
    fy_23_bus.shape,
    list(fy_23_bus.columns),
    fy_24_bus.shape,
    list(fy_24_bus.columns),
)

(40, 8)

['transit_agency',
 'project_title',
 'project_description',
 'new_project_type',
 'total_cost',
 'bus_count',
 'prop_type',
 'bus_size_type']

(42, 8)

['transit_agency',
 'project_title',
 'project_description',
 'new_project_type',
 'total_cost',
 'bus_count',
 'prop_type',
 'bus_size_type']

## Test `.merge()`

In [30]:
fta_all_projects_merge = pd.merge(fy_23_all_projects, fy_24_all_projects, how="outer")

In [31]:
display(fta_all_projects_merge.shape, fta_all_projects_merge.columns)

(247, 11)

Index(['project_sponsor', 'project_title', 'project_description',
       'project_type', 'funding', 'total_bus_count', 'extracted_prop_type',
       'extracted_bus_size', 'agency_name', 'funding_amount',
       'number_of_buses_'],
      dtype='object')

In [32]:
fta_bus_merge = pd.merge(fy_23_bus, fy_24_bus, how="outer")

In [33]:
display(fta_bus_merge.shape, fta_bus_merge.columns)

(82, 8)

Index(['transit_agency', 'project_title', 'project_description',
       'new_project_type', 'total_cost', 'bus_count', 'prop_type',
       'bus_size_type'],
      dtype='object')

# Comparing to initial fta parquet

In [34]:
old_data = pd.read_parquet(f"{_bus_cost_utils.GCS_PATH}clean_fta_bus_only.parquet")

In [35]:
display(
    old_data.shape,
    fta_bus_merge.shape,
    list(old_data.columns),
    list(fta_bus_merge.columns),
)

(43, 8)

(82, 8)

['project_sponsor',
 'project_title',
 'new_prop_type_finder',
 'new_bus_size_type',
 'description',
 'new_project_type',
 'funding',
 'bus_count']

['transit_agency',
 'project_title',
 'project_description',
 'new_project_type',
 'total_cost',
 'bus_count',
 'prop_type',
 'bus_size_type']

In [36]:
old_data.head()

,project_sponsor,project_title,new_prop_type_finder,new_bus_size_type,description,new_project_type,funding,bus_count
0,AUTORIDAD METROPOLITANA DE AUTOBUSES (PRMBA),Puerto Rico Initiative Minimizing Emissions Plus (PRIME+): Continue Transition of Bus Fleet by Purchasing Eight Zero-Emission Transit Buses as an Initiative to bring Sustainable Transit for PR.,electric (not specified),not specified,"The Metropolitan Bus Authority will receive funding to buy new battery-electric buses, which they will maintain using a solar power charging station that was previously supported with federal funds. The buses will operate on high-visibility routes, reduce greenhouse gas emissions, improve reliability and lessen dependence on external energy sources.",bus only,10000000,8
1,Cape Fear Public Transportation Authority,Wave Transit Low Emissions Replacement Vehicles,CNG,not specified,Wave Transit will receive funding to buy compressed natural gas buses for the Wilmington region. These low emission buses will replace older diesel vehicles helping to improve air quality.,bus only,2860250,5
2,"Central Oklahoma Transportation and Parking Authority (COTPA), dba EMBARK","COTPA, dba EMBARK Elimination of Fixed Route Diesel Buses",CNG,not specified,"The Central Oklahoma Transportation and Parking Authority's EMBARK transit will receive funding to buy compressed natural gas buses, replacing the last remaining diesel buses in EMBARK's fixed-route fleet. The project will help carry out EMBARK's 2022 OKC Moves transit improvement plan to reduce emissions and service interruptions, extend service and ensure customers get to their destinations safely and on time.",bus only,4278772,9
3,Champaign-Urbana Mass Transit District,MTD 40-Foot Hybrid Replacement Buses,low emission (hybrid),not specified,"The Champaign-Urbana Mass Transit District will receive funding to buy hybrid-electric replacement buses to replace older buses for residents using bus services in the cities of Champaign and Urbana, the Village of Savoy, and on the University of Illinois campus. The project will improve efficiency and service reliability and help reduce greenhouse gas emissions.",bus only,6635394,10
4,City of Beaumont,Beaumont Municipal Transit Zips to Improve Low Emission Fleet,CNG,not specified,Beaumont Municipal Transit will receive funding to buy compressed natural gas buses to replace older buses that have exceeded their useful life. The buses will help improve safety and reliability as well as air quality.,bus only,2819460,5


# Final outputs

In [37]:
fta_all_projects_merge

fta_bus_merge

,transit_agency,project_title,project_description,new_project_type,total_cost,bus_count,prop_type,bus_size_type
0,Virginia Department of Rail and Public Transportation (DRPT),VA Rural Transit Asset Management and Equity Program,The Virginia Department of Rail and Public Transportation will receive funding to replace the oldest buses operated by 11 rural transit providers. The project includes the purchase of a few dozen replacement buses that are vital to reducing a backlog of adequate vehicles.,Bus,4690010.0,39,mix (low emission),not specified
1,South Carolina Department of Transportation on behalf of 25 rural transit providers,SCDOT Vehicle Replacement Project,The South Carolina Department of Transportation will receive funding to buy new replacement vehicles for 25 rural transit agencies throughout the state. This project will improve transit service and reliability for residents in rural South Carolina communities.,Bus,15423904.0,160,not specified,not specified
2,South Dakota Department of Transportation on behalf of ARL and CTWSI,Replacement of Aberdeen Ride Line fleet buses Replacement and Expansion of Community Transit of Watertown/Sisseton Inc. fleet buses,"The South Dakota Department of Transportation, on behalf of Aberdeen Ride Line and Community Transit of Watertown/Sisseton Inc., will receive funding for nine new alternative, ethanol-fueled buses to reduce greenhouse gas emissions in the city of Aberdeen and 15 counties. The new buses will improve safety and reliability and reduce maintenance costs while also providing a more comfortable passenger ride.",Bus,1006750.0,9,not specified,not specified
3,Madison County Mass Transit District,Heavy Duty 40-Foot Bus Replacement,"The Madison County Mass Transit District will receive funding to buy buses. This project will improve transit service and reliability for residents in Madison County, Illinois.",Bus,1080000.0,2,not specified,not specified
4,State of California on behalf of Kern Regional Transit,Purchase of Fifteen (15) Replacement Cutaway Buses,"The State of California, on behalf of Kern Regional Transit, will receive funding to buy cutaway vehicles to replace older vehicles that have exceeded their useful life. The vehicles will be used for both fixed route and paratransit services.",Bus,2932500.0,15,not specified,cutaway
5,The Colorado Department of Transportation (CDOT) on behalf of San Miguel Authority for Regional Transportation,SMART Vanpool Vehicle Replacement Project,"The Colorado Department of Transportation (CDOT) on behalf of the San Miguel Authority for Regional Transportation in southwest Colorado will receive funding to buy vans to replace older ones. The project will enable the agency to improve its reliability and attract riders in San Miguel, Dolores, Montrose and Ouray counties.",Bus,233760.0,3,not specified,not specified
6,Illinois Department of Transportation on behalf of 33 subrecipients,Illinois DOT Statewide Paratransit Vehicle Replacements,The Illinois Department of Transportation will receive funding to buy cutaway paratransit buses and vans to replace older vehicles for urban and rural public transportation providers throughout Illinois. This project will enhance transit access and mobility for rural communities around the state.,Bus,12600000.0,134,not specified,cutaway
7,Rhode Island Public Transit Authority,RIPTA Hybrid Bus Upgrade,"The Rhode Island Public Transit Authority will receive funding to buy hybrid electric buses to replace older diesel buses. The new buses will reduce greenhouse gas emissions, improve air quality in disadvantaged communities, and enable RIPTA to improve operations and maintenance.",Bus,5000000.0,25,low emission (hybrid),not specified
8,Comanche Nation,Comanche Nation Transit Low/No Project 2023,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new technology. The no-emission vehicles will help improve air quality and fill transportation 

# Script Draft


In [38]:
def clean_fta_columns() -> tuple:
    """
    Updated to read in FTA REST server data. Now reads in fiscal year 23 and 24 data.
    Read in data, renames columns, update specific values.
    """
    fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

    fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

    fy_24_data = to_snakecase(gpd.read_file(fy24))
    fy_23_data = to_snakecase(gpd.read_file(fy23))

    # cleaning fy23 data
    fy_23_data = fy_23_data.assign(
        extracted_prop_type=fy_23_data["project_description"].apply(
            _bus_cost_utils.new_prop_finder
        ),
        extracted_bus_size=fy_23_data["project_description"].apply(
            _bus_cost_utils.new_bus_size_finder
        ),
        fy="fy23",
    )

    fy_23_data["total_bus_count"] = fy_23_data[
        ["traditional_buses", "low_emission_buses", "zero_emission_buses"]
    ].sum(axis=1)
    fy_23_data["funding"] = (
        fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")
    )
    fy_23_data["zero_emission_buses"] = fy_23_data["zero_emission_buses"].astype(
        "int64"
    )

    col_list_23 = [
        "project_sponsor",
        "project_title",
        "project_description",
        "project_type",
        "funding",
        "total_bus_count",
        "extracted_prop_type",
        "extracted_bus_size",
    ]

    fy_23_bus = fy_23_data[(fy_23_data["project_type"] == "Bus")][col_list_23]

    fy_23_all_projects = fy_23_data[col_list_23]

    # cleaning the "not specified" rows
    _bus_cost_utils.col_row_updater(
        fy_23_bus,
        "project_title",
        "VA Rural Transit Asset Management and Equity Program",
        "extracted_prop_type",
        "mix (low emission)",
    )

    # cleaning fy24 data
    fy_24_data = fy_24_data.assign(
        extracted_prop_type=fy_24_data["project_description"].apply(
            _bus_cost_utils.new_prop_finder
        ),
        extracted_bus_size=fy_24_data["project_description"].apply(
            _bus_cost_utils.new_bus_size_finder
        ),
        fy="fy24",
    )

    col_list_24 = [
        "agency_name",
        "project_title",
        "project_description",
        "project_type",
        "funding_amount",
        "number_of_buses_",
        "extracted_prop_type",
        "extracted_bus_size",
    ]

    project_val = ["Vehicle ", "Vehicle"]

    prop_vals = [
        "Battery electric",
        "Hydrogen fuel cell",
        "Battery electric | Hydrogen fuel cell",
        "Battery electric ",
    ]

    fy_24_bus = fy_24_data[col_list_24][(fy_24_data["project_type"].isin(project_val))]

    fy_24_all_projects = fy_24_data[col_list_24]

    _bus_cost_utils.col_row_updater(
        fy_24_bus, "funding_amount", "2894131.0", "extracted_prop_type", "BEB"
    )
    _bus_cost_utils.col_row_updater(
        fy_24_bus, "funding_amount", "14415095.0", "extracted_prop_type", "BEB"
    )
    _bus_cost_utils.col_row_updater(
        fy_24_bus, "funding_amount", "18112632.0", "extracted_prop_type", "BEB"
    )

    # cleaning before merging
    col_dict = {
        "project_sponsor": "transit_agency",
        "agency_name": "transit_agency",
        "project_type": "new_project_type",
        "funding": "total_cost",
        "funding_amount": "total_cost",
        "total_bus_count": "bus_count",
        "number_of_buses_": "bus_count",
        "extracted_prop_type": "prop_type",
        "extracted_bus_size": "bus_size_type",
    }

    fy_23_bus = fy_23_bus.rename(columns=col_dict)

    fy_24_bus = fy_24_bus.rename(columns=col_dict)

    # merge
    fta_all_projects_merge = pd.merge(
        fy_23_all_projects, fy_24_all_projects, how="outer"
    )
    fta_bus_merge = pd.merge(fy_23_bus, fy_24_bus, how="outer")

    return fta_all_projects_merge, fta_bus_merge

## Running function, unpackign the tuple

In [39]:
all_projects, just_bus = clean_fta_columns()

/tmp/ipykernel_1612/1768578308.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fy_23_data["funding"].str.replace("$", "").str.replace(",", "").astype("int64")


In [40]:
display(all_projects.shape, just_bus.shape)

(247, 11)

(82, 8)

In [54]:
# what does data look like
col_dict = {
        "funding": "total_cost",
        "grant_recipient": "transit_agency",
        "new_bus_size": "bus_size_type",
        "new_bus_size_type": "bus_size_type",
        "new_prop_type": "prop_type",
        "new_prop_type_finder": "prop_type",
        "ordering_agency_name": "transit_agency",
        "purchase_order_number": "ppno",
        "quantity": "bus_count",
        "total_project_cost": "total_cost",
        "project_sponsor": "transit_agency",
    }

    # reading in data
    # bus only projects for each datase
fta = pd.read_parquet(f"{_bus_cost_utils.GCS_PATH}clean_fta_bus_only.parquet")
tircp = pd.read_parquet(f"{_bus_cost_utils.GCS_PATH}clean_tircp_bus_only.parquet")
dgs = pd.read_parquet(f"{_bus_cost_utils.GCS_PATH}clean_dgs_bus_only_w_options.parquet")
    
    # adding new column to identify source
fta["source"] = "fta"
tircp["source"] = "tircp"
dgs["source"] = "dgs"

    # using .replace() with dictionary to update column names
fta2 = fta.rename(columns=col_dict)
tircp2 = tircp.rename(columns=col_dict)
dgs2 = dgs.rename(columns=col_dict)

In [57]:
display(list(fta2.columns), list(tircp2.columns), list(dgs2.columns))

['transit_agency',
 'project_title',
 'project_description',
 'new_project_type',
 'total_cost',
 'bus_count',
 'prop_type',
 'bus_size_type',
 'source']

['transit_agency',
 'ppno',
 'prop_type',
 'bus_size_type',
 'project_description',
 'new_project_type',
 'total_cost',
 'bus_count',
 'source']

['transit_agency',
 'ppno',
 'bus_count',
 'prop_type',
 'bus_size_type',
 'source',
 'total_cost']

In [63]:
merge1 = pd.merge(
    fta2,
    tircp2,
    on=[
        "transit_agency",
        "prop_type",
        "bus_size_type",
        "project_description",
        "total_cost",
        "bus_count",
        "source",
        "new_project_type",
    ],
    how="outer",
)

In [64]:
merge1

,transit_agency,project_title,project_description,new_project_type,total_cost,bus_count,prop_type,bus_size_type,source,ppno
0,Virginia Department of Rail and Public Transportation (DRPT),VA Rural Transit Asset Management and Equity Program,The Virginia Department of Rail and Public Transportation will receive funding to replace the oldest buses operated by 11 rural transit providers. The project includes the purchase of a few dozen replacement buses that are vital to reducing a backlog of adequate vehicles.,Bus,4690010.0,39.0,mix (low emission),not specified,fta,NaN
1,South Carolina Department of Transportation on behalf of 25 rural transit providers,SCDOT Vehicle Replacement Project,The South Carolina Department of Transportation will receive funding to buy new replacement vehicles for 25 rural transit agencies throughout the state. This project will improve transit service and reliability for residents in rural South Carolina communities.,Bus,15423904.0,160.0,not specified,not specified,fta,NaN
2,South Dakota Department of Transportation on behalf of ARL and CTWSI,Replacement of Aberdeen Ride Line fleet buses Replacement and Expansion of Community Transit of Watertown/Sisseton Inc. fleet buses,"The South Dakota Department of Transportation, on behalf of Aberdeen Ride Line and Community Transit of Watertown/Sisseton Inc., will receive funding for nine new alternative, ethanol-fueled buses to reduce greenhouse gas emissions in the city of Aberdeen and 15 counties. The new buses will improve safety and reliability and reduce maintenance costs while also providing a more comfortable passenger ride.",Bus,1006750.0,9.0,not specified,not specified,fta,NaN
3,Madison County Mass Transit District,Heavy Duty 40-Foot Bus Replacement,"The Madison County Mass Transit District will receive funding to buy buses. This project will improve transit service and reliability for residents in Madison County, Illinois.",Bus,1080000.0,2.0,not specified,not specified,fta,NaN
4,State of California on behalf of Kern Regional Transit,Purchase of Fifteen (15) Replacement Cutaway Buses,"The State of California, on behalf of Kern Regional Transit, will receive funding to buy cutaway vehicles to replace older vehicles that have exceeded their useful life. The vehicles will be used for both fixed route and paratransit services.",Bus,2932500.0,15.0,not specified,cutaway,fta,NaN
5,The Colorado Department of Transportation (CDOT) on behalf of San Miguel Authority for Regional Transportation,SMART Vanpool Vehicle Replacement Project,"The Colorado Department of Transportation (CDOT) on behalf of the San Miguel Authority for Regional Transportation in southwest Colorado will receive funding to buy vans to replace older ones. The project will enable the agency to improve its reliability and attract riders in San Miguel, Dolores, Montrose and Ouray counties.",Bus,233760.0,3.0,not specified,not specified,fta,NaN
6,Illinois Department of Transportation on behalf of 33 subrecipients,Illinois DOT Statewide Paratransit Vehicle Replacements,The Illinois Department of Transportation will receive funding to buy cutaway paratransit buses and vans to replace older vehicles for urban and rural public transportation providers throughout Illinois. This project will enhance transit access and mobility for rural communities around the state.,Bus,12600000.0,134.0,not specified,cutaway,fta,NaN
7,Rhode Island Public Transit Authority,RIPTA Hybrid Bus Upgrade,"The Rhode Island Public Transit Authority will receive funding to buy hybrid electric buses to replace older diesel buses. The new buses will reduce greenhouse gas emissions, improve air quality in disadvantaged communities, and enable RIPTA to improve operations and maintenance.",Bus,5000000.0,25.0,low emission (hybrid),not specified,fta,NaN
8,Comanche Nation,Comanche Nation Transit Low/No Project 2023,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new

In [65]:
# mergeing merge1 and dgs2
merge2 = pd.merge(
    merge1,
    dgs2,
    on=[
        "transit_agency",
        "prop_type",
        "bus_size_type",
        "total_cost",
        "bus_count",
        "source",
        "ppno",
    ],
    how="outer",
)

In [66]:
merge2

,transit_agency,project_title,project_description,new_project_type,total_cost,bus_count,prop_type,bus_size_type,source,ppno
0,Virginia Department of Rail and Public Transportation (DRPT),VA Rural Transit Asset Management and Equity Program,The Virginia Department of Rail and Public Transportation will receive funding to replace the oldest buses operated by 11 rural transit providers. The project includes the purchase of a few dozen replacement buses that are vital to reducing a backlog of adequate vehicles.,Bus,4690010.0,39.0,mix (low emission),not specified,fta,NaN
1,South Carolina Department of Transportation on behalf of 25 rural transit providers,SCDOT Vehicle Replacement Project,The South Carolina Department of Transportation will receive funding to buy new replacement vehicles for 25 rural transit agencies throughout the state. This project will improve transit service and reliability for residents in rural South Carolina communities.,Bus,15423904.0,160.0,not specified,not specified,fta,NaN
2,South Dakota Department of Transportation on behalf of ARL and CTWSI,Replacement of Aberdeen Ride Line fleet buses Replacement and Expansion of Community Transit of Watertown/Sisseton Inc. fleet buses,"The South Dakota Department of Transportation, on behalf of Aberdeen Ride Line and Community Transit of Watertown/Sisseton Inc., will receive funding for nine new alternative, ethanol-fueled buses to reduce greenhouse gas emissions in the city of Aberdeen and 15 counties. The new buses will improve safety and reliability and reduce maintenance costs while also providing a more comfortable passenger ride.",Bus,1006750.0,9.0,not specified,not specified,fta,NaN
3,Madison County Mass Transit District,Heavy Duty 40-Foot Bus Replacement,"The Madison County Mass Transit District will receive funding to buy buses. This project will improve transit service and reliability for residents in Madison County, Illinois.",Bus,1080000.0,2.0,not specified,not specified,fta,NaN
4,State of California on behalf of Kern Regional Transit,Purchase of Fifteen (15) Replacement Cutaway Buses,"The State of California, on behalf of Kern Regional Transit, will receive funding to buy cutaway vehicles to replace older vehicles that have exceeded their useful life. The vehicles will be used for both fixed route and paratransit services.",Bus,2932500.0,15.0,not specified,cutaway,fta,NaN
5,The Colorado Department of Transportation (CDOT) on behalf of San Miguel Authority for Regional Transportation,SMART Vanpool Vehicle Replacement Project,"The Colorado Department of Transportation (CDOT) on behalf of the San Miguel Authority for Regional Transportation in southwest Colorado will receive funding to buy vans to replace older ones. The project will enable the agency to improve its reliability and attract riders in San Miguel, Dolores, Montrose and Ouray counties.",Bus,233760.0,3.0,not specified,not specified,fta,NaN
6,Illinois Department of Transportation on behalf of 33 subrecipients,Illinois DOT Statewide Paratransit Vehicle Replacements,The Illinois Department of Transportation will receive funding to buy cutaway paratransit buses and vans to replace older vehicles for urban and rural public transportation providers throughout Illinois. This project will enhance transit access and mobility for rural communities around the state.,Bus,12600000.0,134.0,not specified,cutaway,fta,NaN
7,Rhode Island Public Transit Authority,RIPTA Hybrid Bus Upgrade,"The Rhode Island Public Transit Authority will receive funding to buy hybrid electric buses to replace older diesel buses. The new buses will reduce greenhouse gas emissions, improve air quality in disadvantaged communities, and enable RIPTA to improve operations and maintenance.",Bus,5000000.0,25.0,low emission (hybrid),not specified,fta,NaN
8,Comanche Nation,Comanche Nation Transit Low/No Project 2023,Comanche Nation Transit will receive funds to buy battery-electric vehicles and charging equipment and train workers in the new